In [41]:
import os
import sys
sys.path.append('/workspace/GREAT_ASV_system/train')

import numpy as np
import torch


# import vox_model_bank
import model_bank
from read_data import *
from my_dataloader import *
# from sklearn.metrics import roc_curve


In [42]:
model_path = '/workspace/LOGS_OUTPUT/std_server5/train/resnet34_ASV_5/ckpt/fitlog_best_Voxceleb_O_EER.model'
model_id = 'ResNet_ASV'
model_metric = 'Resnet_ASV_large_margin_annealing'

In [43]:
component_dir = '/workspace/LOGS_OUTPUT/std_server5/TEST'
# PLDA_DIM = 370
# PLDA_DATA_NAME = 'plda_data'
# TEST_DATA_NAME = 'test_data_370'
# PLDA_PARA_NAME = 'plda_para_370'
# SCORING_PLDA_NAME = 'score_plda_370'
# SCORING_COSINE_NAME = 'score_cosine_370'
# AUX_DATA_NAME = 'aux_data_370'
# SCORING_AUX_COSINE = 'score_aux_cosine_370'
# SCORING_AUX_PLDA = 'score_aux_plda_370'

In [44]:
if not os.path.isdir(component_dir):
    os.makedirs(component_dir)

# GPU multiprocess

In [45]:
from multiprocessing import Process, Manager

In [52]:
num_p = 10

In [47]:
train_list = '/workspace/DATASET/server9/STD_VOX_EGS/VOX_TRIAL_INFO/trials_fb.csv'
train_data = PickleDataSet(train_list)

In [53]:
train_data_len = len(train_data)
num_per_process = (train_data_len // num_p) + 1

In [54]:
num_per_process

15352

In [55]:
manager = Manager()
class_list_new_m = manager.list()

In [56]:
data_m = []
# class_list_new_m = []
for i in range(num_p):
    data = torch.utils.data.Subset(train_data, np.arange(i*num_per_process, min((i+1)*num_per_process, train_data_len)))
    data_m.append(data)
    class_list_new_m.append({})
    print(len(data))

15352
15352
15352
15352
15352
15352
15352
15352
15352
15348


In [57]:
def extract_feature_m(i, train_data, class_list_new_m):
#     train_list = '/Lun0/zhiyong/dataset/vox12_kaldi_train_data/vox12_kaldi_train_data.csv'
    model_settings = {'in_feat': 40, 'emb_size': 256, 'class_num': 5994, 's': 50, 'm': 0.2, 'anneal_steps': 0, 'HistK_len': 0}

    torch.backends.cudnn.benchmark = False
 
    os.environ['CUDA_VISIBLE_DEVICES'] = str(i%2 + 6)

#     torch.cuda.set_device(i%2)
    device = torch.device("cuda:0")

#     train_data = CSVDataSet(train_list)
#     train_dataloader = DataLoader(dataset=train_data, batch_size = 1, shuffle = False, num_workers = 32, pin_memory=False)

    train_dataloader = My_DataLoader(train_data, batch_size=None, shuffle=False, sampler=None,\
    batch_sampler=None, num_workers=8, collate_fn=None,\
    pin_memory=False, drop_last=False, timeout=0,\
    worker_init_fn=None, multiprocessing_context=None)

    model = model_bank.get_model(model_id, model_metric, model_settings, None)
    checkpoint = torch.load(model_path, map_location='cpu')
    model.load_state_dict(checkpoint['model'], strict=True)
    model = model.to(device)
    model.eval()

    class_list_new = {}

    for count, (batch_x, batch_y) in enumerate(train_dataloader):
        torch.cuda.empty_cache()
        batch_x = batch_x.to(device)
        label = batch_y[0].split('-')[0]
        batch_y = torch.tensor([0]).to(device)
#         try:
#             with torch.no_grad():
#                 _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')
#         except:
#             print('Proc', str(i), 'EER:', label)
#             continue
        _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')

        emb = emb.squeeze().data.cpu().numpy()

        if label not in class_list_new.keys():
            class_list_new[label] = emb[None, :]
        else:
            class_list_new[label] = np.append(class_list_new[label], emb[None, :], axis=0)

        if (count+1) % 10000 == 0:
            print('Proc '+ str(i) + ':' + str((count+1) // 10000))
    
    class_list_new_m[i] = class_list_new
    del model, batch_x, batch_y
    torch.cuda.empty_cache()

In [58]:
processes = [Process(target = extract_feature_m, args = (i, data_m[i], class_list_new_m)) for i in range(num_p)]
[p.start() for p in processes]
joined = [p.join() for p in processes]

Using model bank
Using model bank
Using model bank
Using model bankUsing model bank

Using model bank
Using model bank
Using model bank
Using model bank
Using model bank
Proc 1:1
Proc 8:1
Proc 4:1
Proc 3:1
Proc 5:1
Proc 2:1
Proc 6:1
Proc 9:1
Proc 7:1
Proc 0:1


In [59]:
count = 0
for i in class_list_new_m:
    for j in i:
        count += len(i[j])
count

153516

In [16]:
class_list_new = class_list_new_m[0]

for count, this_list in enumerate(class_list_new_m):
    if count == 0:
        continue
    for this_label in this_list:
        if this_label not in class_list_new.keys():
            class_list_new[this_label] = this_list[this_label]
        else:
            class_list_new[this_label] = np.append(class_list_new[this_label], this_list[this_label], axis=0)

In [17]:
count = 0
for i in class_list_new:
    count += len(class_list_new[i])
count

1276888

In [18]:
nanlist = []
for i in class_list_new:
    if np.isnan(class_list_new[i]).any():
        print(i)
        nanlist.append(i)
for i in nanlist:
    class_list_new.pop(i)

In [19]:
count

1276888

In [20]:
len(class_list_new)

7323

In [21]:
import pickle
out = component_dir+'/'+PLDA_DATA_NAME
with open(out, 'wb') as handle:
    pickle.dump(class_list_new, handle)

In [20]:
# class_list_new = {}
# for i in class_list:
#     class_list_new[i[:-1]] = class_list[i]

# Evaluation

In [38]:
# train_list = '/Lun0/zhiyong/dataset/vox1_kaldi_test/vox1_kaldi_test.csv'
train_list = '/Lun0/zhiyong/dataset/trial_data_csv.csv'
test_list = {}
model_settings = {'in_feat': 30, 'emb_size': 512, 'class_num': 7323, 's': 50, 'm': 0.2, 'anneal_steps': 0, 'HistK_len': 0}

torch.backends.cudnn.benchmark = False
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
device = torch.device("cuda:0")

# train_data = CSVDataSet(train_list)
# train_dataloader = DataLoader(dataset=train_data, batch_size = 1, shuffle = False, num_workers = 32, pin_memory=True)
train_data = PickleDataSet(train_list)
train_dataloader = My_DataLoader(train_data, batch_size=None, shuffle=False, sampler=None,\
batch_sampler=None, num_workers=8, collate_fn=None,\
pin_memory=False, drop_last=False, timeout=0,\
worker_init_fn=None, multiprocessing_context=None)

model = train_model_new.get_model(model_id, model_metric, _, model_settings, _)
checkpoint = torch.load(model_path, map_location='cpu')
model.load_state_dict(checkpoint['model'], strict=True)
model = model.to(device)
model.eval()

for count, (batch_x, batch_y) in enumerate(train_dataloader):
    batch_x = batch_x.to(device)
    label = batch_y[0]

    batch_y = torch.tensor([0]).to(device)
    
    with torch.no_grad():
        _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')

    emb = emb.squeeze().data.cpu().numpy()
    
    if label not in test_list.keys():
        test_list[label] = emb[None, :]
    else:
        print('repeat eer:', label)
        break
#         test_list[label] = np.append(test_list[label], emb[None, :], axis=0)
    
    if (count+1) % 500 == 0:
        print(count+1)

del model, batch_x, batch_y
torch.cuda.empty_cache()

Using new training model bank
500
1000
1500
2000
2500
3000
3500
4000
4500


In [39]:
count+1

4874

In [40]:
len(test_list)

4874

In [41]:
import pickle
out = component_dir+'/'+TEST_DATA_NAME
with open(out, 'wb') as handle:
    pickle.dump(test_list, handle)

# Scoring

In [47]:
# Cosine Scoring

In [48]:
trail_path = '/Lun0/zhiyong/dataset/vox1_kaldi_test/trials'
score_out_path = component_dir+'/'+SCORING_COSINE_NAME

In [49]:
import pickle
with open(component_dir+'/'+TEST_DATA_NAME, 'rb') as handle:
    test_list = pickle.load(handle)

In [50]:
for i in test_list:
    test_list[i] = (1.0 / np.linalg.norm(test_list[i])) * test_list[i]

In [51]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            if count == 0:
                print(line)
#                 continue
            enroll_emb = test_list[line.split(' ')[0][:-4]].squeeze()
            test_emb = test_list[line.split(' ')[1][:-4]].squeeze()
            
            cosine = np.dot(enroll_emb, test_emb)
            
            of.write(line.split(' ')[0]+' '+line.split(' ')[1]+' '+str(cosine)+'\n')
            
            if (count+1) % 5000 == 0:
                print(count+1)

id10270-x6uYqmx31kE-00001.wav id10270-8jEAjG6SegY-00008.wav target

5000
10000
15000
20000
25000
30000
35000
